<a href="https://colab.research.google.com/github/RanieryAV/Topicos-II-Primeiro-Trabalho/blob/main/Arvore-Geradora-Minima/Heap%20de%20Fibonacci/Heap_de_Fibonacci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import pickle
import tracemalloc
import timeit
from time import time_ns
import psutil
import os
import numpy as np

In [2]:
!pip install fibheap

  Preparing metadata (setup.py) ... done
  Created wheel for fibheap: filename=fibheap-0.2.1-py3-none-any.whl size=5867 sha256=8c937a5280df52d641642583aa18d0ceea4a0727532e0f3a97cf0d5785e7ea5f
  Stored in directory: /root/.cache/pip/wheels/91/4b/35/55d7843661b9e8984d3c8863b7bbc00b5c62c30717d08418c0
Successfully built fibheap


In [3]:
from fibheap import *

###Dados

In [ ]:
arquivos1 = ['/content/dados/alue2087.stp','/content/dados/alue2105.stp','/content/dados/alue3146.stp','/content/dados/alue5067.stp','/content/dados/alue5345.stp','/content/dados/alue5623.stp','/content/dados/alue5901.stp','/content/dados/alue6179.stp','/content/dados/alue6457.stp','/content/dados/alue6735.stp','/content/dados/alue6951.stp','/content/dados/alue7065.stp','/content/dados/alue7066.stp','/content/dados/alue7080.stp', '/content/dados/alue7229.stp']

In [ ]:
arquivos2 = ['/content/dados/alut0787.stp','/content/dados/alut0805.stp','/content/dados/alut1181.stp','/content/dados/alut2010.stp','/content/dados/alut2288.stp','/content/dados/alut2566.stp','/content/dados/alut2610.stp','/content/dados/alut2625.stp','/content/dados/alut2764.stp']

In [ ]:
arquivos3 = ['/content/dados/dmxa0296.stp','/content/dados/dmxa0368.stp','/content/dados/dmxa0454.stp','/content/dados/dmxa0628.stp','/content/dados/dmxa0734.stp','/content/dados/dmxa0848.stp','/content/dados/dmxa0903.stp','/content/dados/dmxa1010.stp','/content/dados/dmxa1109.stp','/content/dados/dmxa1200.stp','/content/dados/dmxa1304.stp','/content/dados/dmxa1516.stp','/content/dados/dmxa1721.stp','/content/dados/dmxa1801.stp']

####Listas com as quantidades de vértices e arestas de cada base

#####Vertices

In [61]:
vertices_alue = [1244, 1220, 3626, 3524, 5179, 4472, 11543, 3372, 393, 4119, 2818, 34046, 6405, 34479, 940]
vertices_alut = [1160, 966, 3041, 6104, 9070, 5021, 33901, 36711, 387]
vertices_dmxa = [233, 2050, 1848, 169, 663, 499, 632, 3983, 343, 770, 298, 720, 1005, 2333]

#####Arestas

In [62]:
arestas_alue = [1971, 1858, 5869, 5560, 8165, 6938, 18429, 5213, 6137, 6696, 4419, 54841, 10454, 55494, 1474]
arestas_alut = [2089, 1666, 5693, 11011, 16595, 9055, 62816, 68117, 626]
arestas_dmxa = [386, 3676, 3286, 280, 1154, 861, 1087, 7108, 559, 1383, 503, 1269, 1731, 4137]

##Função do algoritmo de Prim

In [ ]:
def prim(graph, start_node):
    mst = []  # Inicializa a árvore geradora mínima como uma lista vazia
    visited = set([start_node])  # Inicializa o conjunto de nós visitados com o nó inicial
    fib_heap = makefheap()  # Cria um heap de Fibonacci vazio

    # Cria uma lista de arestas do nó inicial e seus custos
    edges = [
        (cost, start_node, to)
        for to, cost in graph[start_node].items()
    ]

    # Insere todas as arestas no heap de Fibonacci com o custo como chave
    for edge in edges:
        fheappush(fib_heap, edge)

    # Continua enquanto houver arestas no heap de Fibonacci
    while fib_heap.min is not None:
        # Obtém e remove a aresta de menor custo do heap de Fibonacci
        cost, frm, to = fheappop(fib_heap)

        # Se o nó 'to' ainda não foi visitado
        if to not in visited:
            visited.add(to)  # Adiciona o nó 'to' ao conjunto de nós visitados
            mst.append((frm, to, cost))  # Adiciona a aresta à árvore geradora mínima

            # Para cada nó adjacente ao nó 'to'
            for to_next, cost2 in graph[to].items():
                # Se o nó adjacente ainda não foi visitado
                if to_next not in visited:
                    # Adiciona a aresta ao heap de Fibonacci
                    fheappush(fib_heap, (cost2, to, to_next))

    return mst # Retorna a árvore geradora mínima


##Função para ler o arquivo .stp e traduzir para uma estrutura de dicionário representando um grafo

In [ ]:
def read_stp(filename):
    graph = {} #graph será um dicionário cujas chaves são os números dos nós e os valores são outros dicionários que por sua vez têm como chaves os nós ligados aos nós das chaves mais externas e seus valores sendo os pesos das arestas que os interligam
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('E '): #percorre as linhas do arquivo manipulando apenas as que iniciam com "E " indicando ser uma aresta do grafo
                data = line.strip().split()
                node1 = int(data[1])
                node2 = int(data[2])
                cost = float(data[3])
                if node1 not in graph:
                    graph[node1] = {}
                if node2 not in graph:
                    graph[node2] = {}
                graph[node1][node2] = cost
                graph[node2][node1] = cost
    return graph

##Função principal que usa as anteriores para ler o arquivo de entrada, modelar o grafo e retornar a MST

In [ ]:
def main():
    filename = '/content/dados/dmxa0296.stp'
    graph = read_stp(filename)
    mst, _ = prim(graph, 1)
    print(mst) #A Mst é retornada como uma lista de tuplas, cada tupla contém a aresta que é representada pelos seus vértices interligados e respectivo peso

In [ ]:
if __name__ == "__main__":
    main()

[(1, 2, 5.0), (1, 14, 13.0), (14, 15, 5.0), (15, 27, 13.0), (27, 26, 5.0), (27, 28, 5.0), (28, 29, 5.0), (29, 30, 5.0), (30, 31, 5.0), (31, 32, 5.0), (32, 33, 5.0), (33, 34, 5.0), (27, 40, 13.0), (40, 39, 5.0), (40, 41, 5.0), (41, 42, 5.0), (42, 43, 5.0), (43, 44, 5.0), (44, 45, 5.0), (45, 46, 5.0), (46, 47, 5.0), (47, 48, 5.0), (48, 49, 5.0), (49, 50, 5.0), (50, 51, 5.0), (51, 52, 5.0), (29, 16, 13.0), (16, 17, 5.0), (17, 18, 5.0), (18, 19, 5.0), (19, 20, 5.0), (20, 21, 5.0), (16, 3, 13.0), (3, 4, 5.0), (4, 5, 5.0), (5, 6, 5.0), (6, 7, 5.0), (7, 8, 5.0), (8, 9, 5.0), (9, 10, 5.0), (10, 11, 5.0), (11, 12, 5.0), (12, 13, 5.0), (10, 22, 13.0), (22, 23, 5.0), (23, 24, 5.0), (24, 25, 5.0), (22, 35, 13.0), (35, 36, 5.0), (36, 37, 5.0), (37, 38, 5.0), (39, 53, 13.0), (53, 54, 5.0), (54, 55, 5.0), (55, 56, 5.0), (56, 57, 5.0), (57, 58, 5.0), (58, 59, 5.0), (47, 60, 13.0), (60, 61, 5.0), (61, 62, 5.0), (62, 63, 5.0), (63, 64, 5.0), (64, 65, 5.0), (53, 66, 13.0), (66, 67, 5.0), (67, 68, 5.0), (

##Funções para automatizar o treino e plotagem de gráficos

In [ ]:
def fheap_Auto(arquivos):
  memoryList = [] #Lista das memórias usadas nas execuções para cada entrada
  timeList = [] #Lista dos tempos decorridos nas execuções para cada entrada
  tracemalloc.start() #inicia a medição de memória
  for arquivo in (arquivos): #percorre todos os arquivos passados como entrada
    graph = read_stp(arquivo) #traduz o grafo presente no arquivo

    start_time = time_ns() #inicia a contagem do tempo da execução atual
    mst = prim(graph, 1) #chama a função de prim
    totalMemory, _ = tracemalloc.get_traced_memory() #obtém o valor da memória total usada na execução da função
    end_time = time_ns() #finaliza a contagem do tempo da execução atual

    tracemalloc.reset_peak() #reseta a medição da memória

    elapsed_time = end_time - start_time #cálculo do tempo total de execução

    memoryList.append(totalMemory) #atualiza a lista de memórias
    timeList.append(elapsed_time) #atualiza a lista de tempos

  return memoryList, timeList

###Função para determinar um tempo mínimo de execução dos testes

In [ ]:
def loop(arquivos):
  fheap_data = {"memoria":{}, "tempo":{}} #dicionário para guardar os valores médios da medição de tempo e memória

  iteracoes = 0 #para contar o número de iterações possibilitando o cálculo de tempo e memória médios
  tempo_total = 0 #variável de controle do laço para detectar o cumprimento dos 5 segundos mínimos de execução
  tempo_parada = 5*10**9 #constante que representa os 5 segundos (em ns)
  memoria_soma = [0]*len(arquivos) #lista para guardar a soma das memórias gastas em cada iteração
  memorias = [] #Para guardar as médias das memórias
  tempos = [] #Para guardar as médias dos tempos
  tempo_soma = [0]*len(arquivos) #lista para guardar a soma dos tempos decorridos em cada iteração
  tempo_inicial = time_ns() #início da contagem do tempo relativo aos 5 segundos mínimos de execução
  while tempo_total < tempo_parada: #laço para garantir o término das execuções com no mínimo 5 segundos decorridos
    memoria, tempo = fheap_Auto(arquivos) #pega as listas de memória e tempo da iteração corrente
    memoria_soma = [mem1 + mem2 for mem1, mem2 in zip(memoria_soma, memoria)] #Atualiza a lista das somas das memórias
    tempo_soma = [tem1 + tem2 for tem1, tem2 in zip(tempo_soma, tempo)] #Atualiza a lista das somas dos tempos
    tempo_final = time_ns() #ponto de verificação do tempo decorrido do laço enquanto
    tempo_total = tempo_final - tempo_inicial #cálculo do tempo total decorrido
    iteracoes += 1 #incremento do número de iterações

  memorias = [item/iteracoes for item in memoria_soma] #calcula a média das memórias
  tempos = [item/iteracoes for item in tempo_soma] #calcula a média dos tempos

  for arquivo, memoria, tempo in zip(arquivos, memorias, tempos): #Laço para formação do dicionário contendo as informações de memória e tempo médios da base testada
    fheap_data['memoria'][arquivo] = memoria
    fheap_data['tempo'][arquivo] = tempo

  return fheap_data, iteracoes

dic_fheap, iteracoes = loop(arquivos2) #execução da função de teste que garante os mínimos 5 segundos de teste

with open(f"fheap_alut({iteracoes}).pkl", "wb") as f:
    # Usa a função dump para salvar o dicionário no arquivo
    pickle.dump(dic_fheap, f)


In [ ]:
dic_fheap

{'memoria': {'/content/dados/alut0787.stp': 577561.0,
  '/content/dados/alut0805.stp': 569552.0,
  '/content/dados/alut1181.stp': 1598952.0,
  '/content/dados/alut2010.stp': 3174877.0,
  '/content/dados/alut2288.stp': 4523306.0,
  '/content/dados/alut2566.stp': 2560728.0,
  '/content/dados/alut2610.stp': 16797639.0,
  '/content/dados/alut2625.stp': 18089440.0,
  '/content/dados/alut2764.stp': 334611.0},
 'tempo': {'/content/dados/alut0787.stp': 97225638.0,
  '/content/dados/alut0805.stp': 50916437.0,
  '/content/dados/alut1181.stp': 215079143.0,
  '/content/dados/alut2010.stp': 548045833.0,
  '/content/dados/alut2288.stp': 540964795.0,
  '/content/dados/alut2566.stp': 192289444.0,
  '/content/dados/alut2610.stp': 2233185105.0,
  '/content/dados/alut2625.stp': 2415203045.0,
  '/content/dados/alut2764.stp': 56033040.0}}

##Tratamento dos dados do gráfico

In [60]:
nome_arquivo = f"fheap_dmxa(6).pkl"
with open(nome_arquivo, 'rb') as arquivo:
    fheap = pickle.load(arquivo)

###Relacionar numero de vértices com tempo

In [7]:
#ALUE
tempos_alue = fheap['tempo']

alue_tempos_vertices_tuplas = []

while tempos_alue:
  alue_tempos_vertices_tuplas.append((vertices_alue.pop(), tempos_alue.popitem()[1]))

alue_tempos_vertices_tuplas.sort()

In [28]:
#ALUT
tempos_alut = fheap['tempo']

alut_tempos_vertices_tuplas = []

while tempos_alut:
  alut_tempos_vertices_tuplas.append((vertices_alut.pop(), tempos_alut.popitem()[1]))

alut_tempos_vertices_tuplas.sort()

In [48]:
#DMXA
tempos_dmxa = fheap['tempo']

dmxa_tempos_vertices_tuplas = []

while tempos_dmxa:
  dmxa_tempos_vertices_tuplas.append((vertices_dmxa.pop(), tempos_dmxa.popitem()[1]))

dmxa_tempos_vertices_tuplas.sort()

###Relacionar numero de vértices com memória

In [13]:
#ALUE
memorias_alue = fheap['memoria']

alue_memorias_vertices_tuplas = []

while memorias_alue:
  alue_memorias_vertices_tuplas.append((vertices_alue.pop(), memorias_alue.popitem()[1]))

alue_memorias_vertices_tuplas.sort()

In [33]:
#ALUT
memorias_alut = fheap['memoria']

alut_memorias_vertices_tuplas = []

while memorias_alut:
  alut_memorias_vertices_tuplas.append((vertices_alut.pop(), memorias_alut.popitem()[1]))

alut_memorias_vertices_tuplas.sort()

In [53]:
#DMXA
memorias_dmxa = fheap['memoria']

dmxa_memorias_vertices_tuplas = []

while memorias_dmxa:
  dmxa_memorias_vertices_tuplas.append((vertices_dmxa.pop(), memorias_dmxa.popitem()[1]))

dmxa_memorias_vertices_tuplas.sort()

###Relacionar numero de arestas com tempo

In [18]:
#ALUE
tempos_alue = fheap['tempo']

alue_tempos_arestas_tuplas = []

while tempos_alue:
  alue_tempos_arestas_tuplas.append((arestas_alue.pop(), tempos_alue.popitem()[1]))

alue_tempos_arestas_tuplas.sort()

In [38]:
#ALUT
tempos_alut = fheap['tempo']

alut_tempos_arestas_tuplas = []

while tempos_alut:
  alut_tempos_arestas_tuplas.append((arestas_alut.pop(), tempos_alut.popitem()[1]))

alut_tempos_arestas_tuplas.sort()

In [58]:
#DMXA
tempos_dmxa = fheap['tempo']

dmxa_tempos_arestas_tuplas = []

while tempos_dmxa:
  dmxa_tempos_arestas_tuplas.append((arestas_dmxa.pop(), tempos_dmxa.popitem()[1]))

dmxa_tempos_arestas_tuplas.sort()

###Relacionar numero de arestas com memória

In [23]:
#ALUE
memorias_alue = fheap['memoria']

alue_memorias_arestas_tuplas = []

while memorias_alue:
  alue_memorias_arestas_tuplas.append((arestas_alue.pop(), memorias_alue.popitem()[1]))

alue_memorias_arestas_tuplas.sort()

In [43]:
#ALUT
memorias_alut = fheap['memoria']

alut_memorias_arestas_tuplas = []

while memorias_alut:
  alut_memorias_arestas_tuplas.append((arestas_alut.pop(), memorias_alut.popitem()[1]))

alut_memorias_arestas_tuplas.sort()

In [63]:
#DMXA
memorias_dmxa = fheap['memoria']

dmxa_memorias_arestas_tuplas = []

while memorias_dmxa:
  dmxa_memorias_arestas_tuplas.append((arestas_dmxa.pop(), memorias_dmxa.popitem()[1]))

dmxa_memorias_arestas_tuplas.sort()

###Plotar gráficos

In [8]:
import plotly.express as px

####Relacionar numero de vertices com tempo

In [9]:
#ALUE
x = [item[0] for item in alue_tempos_vertices_tuplas]
y = [item[1] for item in alue_tempos_vertices_tuplas]

# Criar o gráfico de dispersão
fig = px.scatter(x=x, y=y, title=f"Tempo gasto na execução de Prim usando Heap de Fibonacci (ALUE)",
                 labels={'x': 'Vértices', 'y': 'Tempo (ns)'},
                 log_x=True)

# Personalizar a legenda
fig.update_traces(name='Pontos de Dados')  # Nomeie a série de dados
fig.update_xaxes(tickmode='array', tickvals=x[::2], ticktext=x[::2], tickangle=45)

# Exibir o gráfico
fig.show()

In [29]:
#ALUT
x = [item[0] for item in alut_tempos_vertices_tuplas]
y = [item[1] for item in alut_tempos_vertices_tuplas]

# Criar o gráfico de dispersão
fig = px.scatter(x=x, y=y, title=f"Tempo gasto na execução de Prim usando Heap de Fibonacci (ALUT)",
                 labels={'x': 'Vértices', 'y': 'Tempo (ns)'},
                 log_x=True)

# Personalizar a legenda
fig.update_traces(name='Pontos de Dados')  # Nomeie a série de dados
fig.update_xaxes(tickmode='array', tickvals=x[::2], ticktext=x[::2], tickangle=45)

# Exibir o gráfico
fig.show()

In [49]:
#DMXA
x = [item[0] for item in dmxa_tempos_vertices_tuplas]
y = [item[1] for item in dmxa_tempos_vertices_tuplas]

# Criar o gráfico de dispersão
fig = px.scatter(x=x, y=y, title=f"Tempo gasto na execução de Prim usando Heap de Fibonacci (DMXA)",
                 labels={'x': 'Vértices', 'y': 'Tempo (ns)'},
                 log_x=True)

# Personalizar a legenda
fig.update_traces(name='Pontos de Dados')  # Nomeie a série de dados
fig.update_xaxes(tickmode='array', tickvals=x[::2], ticktext=x[::2], tickangle=45)

# Exibir o gráfico
fig.show()

####Relacionar numero de vertices com memoria

In [14]:
#ALUE
x = [item[0] for item in alue_memorias_vertices_tuplas]
y = [item[1] for item in alue_memorias_vertices_tuplas]

# Criar o gráfico de dispersão
fig = px.scatter(x=x, y=y, title=f"Memória usada na execução de Prim usando Heap de Fibonacci (ALUE)",
                 labels={'x': 'Vértices', 'y': 'Memória (B)'},
                 log_x=True)

# Personalizar a legenda
fig.update_traces(name='Pontos de Dados')  # Nomeie a série de dados
fig.update_xaxes(tickmode='array', tickvals=x[::2], ticktext=x[::2], tickangle=45)

# Exibir o gráfico
fig.show()

In [34]:
#ALUT
x = [item[0] for item in alut_memorias_vertices_tuplas]
y = [item[1] for item in alut_memorias_vertices_tuplas]

# Criar o gráfico de dispersão
fig = px.scatter(x=x, y=y, title=f"Memória usada na execução de Prim usando Heap de Fibonacci (ALUT)",
                 labels={'x': 'Vértices', 'y': 'Memória (B)'},
                 log_x=True)

# Personalizar a legenda
fig.update_traces(name='Pontos de Dados')  # Nomeie a série de dados
fig.update_xaxes(tickmode='array', tickvals=x[::2], ticktext=x[::2], tickangle=45)

# Exibir o gráfico
fig.show()

In [54]:
#DMXA
x = [item[0] for item in dmxa_memorias_vertices_tuplas]
y = [item[1] for item in dmxa_memorias_vertices_tuplas]

# Criar o gráfico de dispersão
fig = px.scatter(x=x, y=y, title=f"Memória usada na execução de Prim usando Heap de Fibonacci (DMXA)",
                 labels={'x': 'Vértices', 'y': 'Memória (B)'},
                 log_x=True)

# Personalizar a legenda
fig.update_traces(name='Pontos de Dados')  # Nomeie a série de dados
fig.update_xaxes(tickmode='array', tickvals=x[::2], ticktext=x[::2], tickangle=45)

# Exibir o gráfico
fig.show()

####Relacionar numero de arestas com tempo

In [19]:
#ALUE
x = [item[0] for item in alue_tempos_arestas_tuplas]
y = [item[1] for item in alue_tempos_arestas_tuplas]

# Criar o gráfico de dispersão
fig = px.scatter(x=x, y=y, title=f"Tempo gasto na execução de Prim usando Heap de Fibonacci (ALUE)",
                 labels={'x': 'Arestas', 'y': 'Tempo (ns)'},
                 log_x=True)

# Personalizar a legenda
fig.update_traces(name='Pontos de Dados')  # Nomeie a série de dados
fig.update_xaxes(tickmode='array', tickvals=x[::2], ticktext=x[::2], tickangle=45)

# Exibir o gráfico
fig.show()

In [39]:
#ALUT
x = [item[0] for item in alut_tempos_arestas_tuplas]
y = [item[1] for item in alut_tempos_arestas_tuplas]

# Criar o gráfico de dispersão
fig = px.scatter(x=x, y=y, title=f"Tempo gasto na execução de Prim usando Heap de Fibonacci (ALUT)",
                 labels={'x': 'Arestas', 'y': 'Tempo (ns)'},
                 log_x=True)

# Personalizar a legenda
fig.update_traces(name='Pontos de Dados')  # Nomeie a série de dados
fig.update_xaxes(tickmode='array', tickvals=x[::2], ticktext=x[::2], tickangle=45)

# Exibir o gráfico
fig.show()

In [59]:
#DMXA
x = [item[0] for item in dmxa_tempos_arestas_tuplas]
y = [item[1] for item in dmxa_tempos_arestas_tuplas]

# Criar o gráfico de dispersão
fig = px.scatter(x=x, y=y, title=f"Tempo gasto na execução de Prim usando Heap de Fibonacci (DMXA)",
                 labels={'x': 'Arestas', 'y': 'Tempo (ns)'},
                 log_x=True)

# Personalizar a legenda
fig.update_traces(name='Pontos de Dados')  # Nomeie a série de dados
fig.update_xaxes(tickmode='array', tickvals=x[::2], ticktext=x[::2], tickangle=45)

# Exibir o gráfico
fig.show()

####Relacionar numero de arestas com memoria

In [24]:
#ALUE
x = [item[0] for item in alue_memorias_arestas_tuplas]
y = [item[1] for item in alue_memorias_arestas_tuplas]

# Criar o gráfico de dispersão
fig = px.scatter(x=x, y=y, title=f"Memória usada na execução de Prim usando Heap de Fibonacci (ALUE)",
                 labels={'x': 'Arestas', 'y': 'Memória (B)'},
                 log_x=True)

# Personalizar a legenda
fig.update_traces(name='Pontos de Dados')  # Nomeie a série de dados
fig.update_xaxes(tickmode='array', tickvals=x[::2], ticktext=x[::2], tickangle=45)

# Exibir o gráfico
fig.show()

In [44]:
#ALUT
x = [item[0] for item in alut_memorias_arestas_tuplas]
y = [item[1] for item in alut_memorias_arestas_tuplas]

# Criar o gráfico de dispersão
fig = px.scatter(x=x, y=y, title=f"Memória usada na execução de Prim usando Heap de Fibonacci (ALUT)",
                 labels={'x': 'Arestas', 'y': 'Memória (B)'},
                 log_x=True)

# Personalizar a legenda
fig.update_traces(name='Pontos de Dados')  # Nomeie a série de dados
fig.update_xaxes(tickmode='array', tickvals=x[::2], ticktext=x[::2], tickangle=45)

# Exibir o gráfico
fig.show()

In [64]:
#DMXA
x = [item[0] for item in dmxa_memorias_arestas_tuplas]
y = [item[1] for item in dmxa_memorias_arestas_tuplas]

# Criar o gráfico de dispersão
fig = px.scatter(x=x, y=y, title=f"Memória usada na execução de Prim usando Heap de Fibonacci (DMXA)",
                 labels={'x': 'Arestas', 'y': 'Memória (B)'},
                 log_x=True)

# Personalizar a legenda
fig.update_traces(name='Pontos de Dados')  # Nomeie a série de dados
fig.update_xaxes(tickmode='array', tickvals=x[::2], ticktext=x[::2], tickangle=45)

# Exibir o gráfico
fig.show()

##Exemplo de como é lido e interpretado o grafo de um arquivo de entrada

In [ ]:
filename = '/content/dados/dmxa0296.stp'
graph = read_stp(filename)

In [ ]:
for to, cost in graph[1].items():
  print(f"to: {to}; cost: {cost}")

to: 2; cost: 5.0
to: 14; cost: 13.0


In [ ]:
heap = makefheap()

In [ ]:
fheappush(heap, (2, 4, 5))

In [ ]:
fheappush(heap, (1, 3, 6))

In [ ]:
fheappush(heap, (3, 5, 9))

In [ ]:
getfheapmin(heap)

(3, 5, 9)

In [ ]:
fheappop(heap)

(2, 4, 5)

In [ ]:
heap.min is None

False